In [ ]:
import pandas as pd

import matplotlib.dates as mdates
import matplotlib.pyplot as plt

In [38]:
model_list = ["arima", "lagllama"]
metrics = ["mae", "rmse", "smape"]
group = ["dataset", "train_size", "horizon"]

for model in model_list:
    results = pd.read_csv(f"../out/{model}_results.csv")
    mean_metrics_df = results.groupby(group)[metrics].agg(["mean", "std"])
    mean_metrics_df = mean_metrics_df.reset_index()
    mean_metrics_df_rounded = round(mean_metrics_df, 3)
    mean_metrics_df_rounded.to_csv(f"../out/{model}_mean_results.csv")

In [ ]:
model_list = ["arima", "lagllama"]

dataset_label = ["weather", "carbon"]
train_size_label = ["small", "large"]
test_size_label = ["small", "large"]
horizons = [5, 50]

In [ ]:

for dataset in dataset_label:
    for train_size in train_size_label:
        for horizon in horizons:
            if horizon == 5:
                test_size = "small"
            elif horizon == 50:
                test_size = "large"
            
            # True values:
            train = pd.read_csv(f"../data/train/{dataset}_train.csv")
            test = pd.read_csv(f"../data/test/{dataset}_test_{test_size}.csv")

            if dataset == "weather":
                train_y, test_y = train["Temperatur"], test["Temperatur"].iloc[:50]
                ylabel = "Temperature"
                title = "Temperature Forecast"
            if dataset == "carbon":
                train_y, test_y = train["Carbon_intensity"], test["Carbon_intensity"].iloc[:50]
                ylabel = "Carbon Intensity"
                title = "Carbon Intensity Forecast"
     
            true_y = pd.concat([train_y, test_y]).reset_index(drop = True)
            true_x = pd.concat([train["Date"], test["Date"]]).iloc[:len(true_y)].reset_index(drop = True)

            forecasts = {}
            for model in model_list:
                 
                # Forecasted values:
                forecast = pd.read_csv(f"../out/{model}_{horizon}_{dataset}.csv")

                max_len = min(len(forecast), len(test))
                forecast_y = forecast["0"].iloc[:max_len].reset_index(drop = True)
                forecast_x = test["Date"].iloc[:max_len].reset_index(drop = True)
                forecasts[model] = (forecast_x, forecast_y)

            # Plot!

            plt.figure(figsize = (14, 6))
            ax = plt.gca()
            ax.plot(true_x, true_y, label = "True", color = "seagreen")

            for model in model_list:
                forecast_x, forecast_y = forecasts[model]
                ax.plot(forecast_x, forecast_y, label = model.upper())

            ax.set_xlabel("Time")
            ax.set_ylabel(ylabel)
            ax.set_title(title)
            ax.legend(loc = "lower right")

            ax.xaxis.set_major_locator(mdates.AutoDateLocator())
            ax.xaxis.set_major_formatter(mdates.DataFormatter("%Y-%m-%d"))

            plt.xticks(rotation = 45)
            plt.tight_layout()
            plt.show()
     